<a href="https://colab.research.google.com/github/pejho/nd00333_AZMLND_Optimizing_a_Pipeline_in_Azure-Starter_Files/blob/master/Copy_of_Copy_of_udacity_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-181440
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-181440


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException



# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
comp_cluster = 'opt-ml-cluster'
try:
    comp_trget = ComputeTarget(workspace=ws,name=comp_cluster)
except ComputeTargetException:  
    comp_confg = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4, min_nodes=1)
comp_trget = ComputeTarget.create(ws, comp_cluster, comp_confg)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{
    '--C' : choice(0.03,0.3,3,10,30),
    '--max_iter' : choice(25,50,75,100)
})

### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=comp_trget,
                      environment = sklearn_env)
### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=40,
    max_concurrent_runs=5,
    policy=policy,
    run_config=src
)
### YOUR CODE HERE ###

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.widgets import RunDetails

hyperdrive_submission = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_submission).show()
hyperdrive_submission.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_0d1653d7-0e2c-4ca0-a4b2-451b68f6c5cc
Web View: https://ml.azure.com/runs/HD_0d1653d7-0e2c-4ca0-a4b2-451b68f6c5cc?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-181440/workspaces/quick-starts-ws-181440&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-05T03:07:08.807208][API][INFO]Experiment created<END>\n""<START>[2022-02-05T03:07:10.512067][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2022-02-05T03:07:10.826378][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_0d1653d7-0e2c-4ca0-a4b2-451b68f6c5cc
Web View: https://ml.azure.com/runs/HD_0d1653d7-0e2c-4ca0-a4b2-451b68f6c5cc?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-181440/workspaces/quick-starts-ws-181440&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_0d1653d7-0e2c-4ca0-a4b2-451b68f6c5cc',
 'target': 'opt-ml-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-05T03:07:08.490339Z',
 'endTimeUtc': '2022-02-05T03:24:47.712464Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4c380ab8-05ad-438e-9586-5edb742a67bd',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1064-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.37.0',
  'space_size': '20',
  'score': '0.9134041476985332',
  'best_child_run_id': 'HD_0d1653d7-0e2c-4ca0-a4b2-451b68f6c5cc_13',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg181440.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_0d1653d7-0e2c-4ca0-a4b2-451b68f6c5cc/azure

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_submission.get_best_run_by_primary_metric()
print('ID : ',best_run.id)
print('Metrics : ',best_run.get_metrics())

final_model = best_run.register_model(model_name = 'hypermodel',model_path = './')

ID :  HD_0d1653d7-0e2c-4ca0-a4b2-451b68f6c5cc_13
Metrics :  {'Regularization Strength:': 30.0, 'Max iterations:': 100, 'Accuracy': 0.9134041476985332}


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
data_link = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
input_data = TabularDatasetFactory.from_delimited_files(path=data_link)

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(input_data)

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    label_column_name='y',
    training_data = input_data,
    compute_target = comp_trget,
    max_concurrent_iterations = 3,
    enable_early_stopping = True,
    n_cross_validations=7)


In [9]:
# Submit your automl run

### YOUR CODE HERE ###
class_automl = exp.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on opt-ml-cluster with default configuration
Running on remote compute: opt-ml-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_f1cc545d-4277-4998-b7a1-891693d4b21b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [10]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_class_automl, best_fit_model = class_automl.get_output()
print(best_fit_model)

joblib.dump(best_fit_model, 'outputs/best_fit_automl_model.joblib')
best_class_automl.register_model(model_name = 'best_automl.pkl', model_path = './')

Package:azureml-automl-runtime, training version:1.38.0, current version:1.37.0
Package:azureml-core, training version:1.38.0, current version:1.37.0
Package:azureml-dataprep, training version:2.26.0, current version:2.25.0
Package:azureml-dataprep-rslex, training version:2.2.0, current version:2.1.0
Package:azureml-dataset-runtime, training version:1.38.0, current version:1.37.0
Package:azureml-defaults, training version:1.38.0, current version:1.37.0
Package:azureml-interpret, training version:1.38.0, current version:1.37.0
Package:azureml-mlflow, training version:1.38.0, current version:1.37.0
Package:azureml-pipeline-core, training version:1.38.0, current version:1.37.0
Package:azureml-responsibleai, training version:1.38.0, current version:1.37.0
Package:azureml-telemetry, training version:1.38.0, current version:1.37.0
Package:azureml-train-automl-client, training version:1.38.0, current version:1.37.0
Package:azureml-train-automl-runtime, training version:1.38.0, current version

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('logisticregression', LogisticRegression(C=51.79474679231202, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('13', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('sgdclassifierwrapper', SGDClassifierWrapper(alpha=7.5510448979591835, class_weight='balanced', eta0=0.001, fit_intercept=True, l1_ratio=0.42857142857142855, learning_rate='constant', loss='modified_huber', max_iter=1000,

Model(workspace=Workspace.create(name='quick-starts-ws-181440', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-181440'), name=best_automl.pkl, id=best_automl.pkl:1, version=1, tags={}, properties={})

In [11]:
comp_trget.delete()


Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

